In [1]:
%%time
import os
import keras
import numpy as np
from keras.datasets import mnist
import sagemaker
import numpy as np
from sklearn.externals import joblib
import pandas as pd
import tensorflow as tf
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


CPU times: user 2.23 s, sys: 168 ms, total: 2.4 s
Wall time: 2.41 s


In [161]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

# inp = bytes(str(exc[0].flatten().tolist()), 'utf-8')
inp = json.dumps(exc[0].flatten().tolist())

response = client.invoke_endpoint(
    EndpointName='sagemaker-tensorflow-scriptmode-2019-11-08-05-56-35-964',
    Body=inp,
    ContentType='application/json',
    Accept='application/json'
)

body = response['Body']
json.load(body)

{'prediction': 198}

## トレーニングデータをS3にアップする

In [2]:
def _load_data(data, n_prev=50):
    docX, docY = [], []
    for i in range(len(data) - n_prev):
        if i == 0:
            continue
        docX.append(data.iloc[i - 1:i + n_prev - 1].values)
        docY.append(data.iloc[i + n_prev].values)
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY


# 学習用とテスト用データを分割、ただし分割する際に_load_data()を適用
def train_test_split(df, test_size=0.1, n_prev=50):
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))
    ntrn = int(ntrn)
    X_train, y_train = _load_data(df.iloc[0:ntrn], n_prev)
    X_test, y_test = _load_data(df.iloc[ntrn:], n_prev)

    return (X_train, y_train), (X_test, y_test)


def inverse_original_scale(x):
    x = scaler.inverse_transform(x.reshape(1, -1))
    x = np.exp(x)
    return x



df_code_date_hourly = pd.read_csv('./code_run_num_hourly.csv')
split_date = lambda x: x.split(" ")[0]
df_code_date_hourly["date"] = list(map(split_date, df_code_date_hourly['timestamp'].values))

# log scaling
df_code_date_hourly['code_num'] = np.log(df_code_date_hourly['demand'])

# MinMaxS scaling
scaler = MinMaxScaler(feature_range=(0, 1))
df_code_date_hourly['code_num'] = scaler.fit_transform(df_code_date_hourly['code_num'].values.reshape(-1, 1))

# 

length_of_sequences = 24
test_size = 0.1

(X_train, y_train), (X_test, y_test) = train_test_split(df_code_date_hourly[['demand']], test_size=test_size,
                                                        n_prev=length_of_sequences)

# row_data = df_code_date_hourly['demand'].values

In [3]:
y_train.shape

(10701, 1)

In [4]:
(_, _), (exc, _) = train_test_split(df_code_date_hourly[['demand']], test_size=test_size,
                                                        n_prev=length_of_sequences)

In [143]:
os.makedirs("./anomaly_data", exist_ok = True)

# np.savez('./data/train', image=X_train, label=y_train)
# np.savez('./data/test', image=X_test, label=y_test)
df_code_date_hourly = pd.read_csv('./code_run_num_hourly.csv')
df_code_date_hourly.to_csv('anomaly_data/train_data.csv')
# np.savez('./data/row_data', row_data=row_data)
joblib.dump(scaler, './anomaly_data/scaler.save')

import sagemaker
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
input_data = sagemaker_session.upload_data(path='./anomaly_data', bucket=bucket_name, key_prefix='dataset/anomaly_detection')
print('Training data is uploaded to: {}'.format(input_data))

Training data is uploaded to: s3://sagemaker-ap-northeast-1-481470706855/dataset/anomaly_detection


### TensorFlowScriptModeコンテナ

In [144]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role


hyper_param = {
    'batch-size': 256,
    'epochs': 50
}

role = get_execution_role()
estimator = TensorFlow(
    entry_point = "./anomaly_data/anomaly_model.py",
    role=role,
    train_instance_count=1,
    train_instance_type="ml.m4.xlarge",
    framework_version="1.12.0",
    py_version='py3',
    script_mode=True,
    hyperparameters=hyper_param
)

estimator.fit(input_data)

2019-11-08 04:10:43 Starting - Starting the training job...
2019-11-08 04:10:44 Starting - Launching requested ML instances......
2019-11-08 04:11:46 Starting - Preparing the instances for training...
2019-11-08 04:12:29 Downloading - Downloading input data...
2019-11-08 04:13:11 Training - Training image download completed. Training in progress..
2019-11-08 04:13:14,669 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-11-08 04:13:14,676 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-08 04:13:15,043 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-08 04:13:15,060 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-11-08 04:13:15,074 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_

9509/9509 [==============================] - 4s 391us/step - loss: 0.0250 - val_loss: 0.0254
Epoch 3/50

9509/9509 [==============================] - 4s 401us/step - loss: 0.0234 - val_loss: 0.0245
Epoch 4/50

9509/9509 [==============================] - 4s 394us/step - loss: 0.0220 - val_loss: 0.0225
Epoch 5/50

9509/9509 [==============================] - 4s 395us/step - loss: 0.0195 - val_loss: 0.0183
Epoch 6/50
9509/9509 [==============================] - 4s 390us/step - loss: 0.0140 - val_loss: 0.0127
Epoch 7/50

9509/9509 [==============================] - 4s 400us/step - loss: 0.0110 - val_loss: 0.0135
Epoch 00007: early stopping
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.21.2 when using version 0.21.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator M

Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
2019-11-08 04:13:51,667 sagemaker-containers INFO     Reporting training SUCCESS

2019-11-08 04:13:58 Uploading - Uploading generated training model
2019-11-08 04:13:58 Completed - Training job completed
Billable seconds: 89


In [145]:
predictor_ex = estimator.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

---------------------------------------------------------------------------------------------------!

In [39]:
exc[0].flatten() + 100

array([ 354,  369,  341,  215,  170,  163,  171,  116,  112,  140,  168,
        264,  292,  359,  365,  248,  327,  830,  838, 1012,  400,  371,
        253,  247])

In [139]:
exc[0]

ops = {
    "true": exc[0].flatten().tolist(),
    "pred": (exc[0].flatten() + 1000).tolist()
}

ops = json.dumps(ops)

In [44]:
ops

'{"true": [254, 269, 241, 115, 70, 63, 71, 16, 12, 40, 68, 164, 192, 259, 265, 148, 227, 730, 738, 912, 300, 271, 153, 147], "pred": [354, 369, 341, 215, 170, 163, 171, 116, 112, 140, 168, 264, 292, 359, 365, 248, 327, 830, 838, 1012, 400, 371, 253, 247]}'

In [84]:
json.load(ops)['true']

AttributeError: 'str' object has no attribute 'read'

In [146]:
predictor_ex.predict(ops)['judgement']

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "{"error": "{\"error\": \"could not preprocess input data: \"{\\\"true\\\": [254, 269, 241, 115, 70, 63, 71, 16, 12, 40, 68, 164, 192, 259, 265, 148, 227, 730, 738, 912, 300, 271, 153, 147], \\\"pred\\\": [1254, 1269, 1241, 1115, 1070, 1063, 1071, 1016, 1012, 1040, 1068, 1164, 1192, 1259, 1265, 1148, 1227, 1730, 1738, 1912, 1300, 1271, 1153, 1147]}\"\"}"}". See https://ap-northeast-1.console.aws.amazon.com/cloudwatch/home?region=ap-northeast-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-tensorflow-scriptmode-2019-11-08-04-10-40-842 in account 481470706855 for more information.

In [118]:
import subprocess

subprocess.check_output("pwd")

b'/home/ec2-user/SageMaker\n'

In [50]:


response = client.invoke_endpoint(
    EndpointName='sagemaker-tensorflow-scriptmode-2019-11-07-03-29-18-520',
    Body=ops,
    ContentType='application/json',
    Accept='application/json'
)

body = response['Body']
json.load(body)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "{"error": "{\"error\": \"could not preprocess input data: {\"true\": [254, 269, 241, 115, 70, 63, 71, 16, 12, 40, 68, 164, 192, 259, 265, 148, 227, 730, 738, 912, 300, 271, 153, 147], \"pred\": [354, 369, 341, 215, 170, 163, 171, 116, 112, 140, 168, 264, 292, 359, 365, 248, 327, 830, 838, 1012, 400, 371, 253, 247]}\"}"}". See https://ap-northeast-1.console.aws.amazon.com/cloudwatch/home?region=ap-northeast-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-tensorflow-scriptmode-2019-11-07-03-29-18-520 in account 481470706855 for more information.

In [99]:
a = [1, 2]
b = [2, 4]
a = np.array(a)
b = np.array(b)
(a - b).mean()

-1.5

### SageMakerTensorFlowServingコンテナで推論エンドポイントのデプロイ

In [29]:
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role
from sagemaker.tensorflow.serving import Model


role = get_execution_role()
model = Model(
    model_data="s3://sagemaker-ap-northeast-1-481470706855/sagemaker-tensorflow-scriptmode-2019-10-30-06-14-48-966/output/model.tar.gz",
    role=role,
    framework_version="1.12.0"
    )



## モデルのデプロイ

In [30]:
predictor = model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

--------------------------------------------------------------------------------------------------!

In [ ]:
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, 'sagemaker-tensorflow-serving')
print(container)

## 推論

In [90]:
def _load_data(data, n_prev=50):
    docX, docY = [], []
    for i in range(len(data) - n_prev):
        if i == 0:
            continue
        docX.append(data.iloc[i - 1:i + n_prev - 1].values)
        docY.append(data.iloc[i + n_prev].values)
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY


# 学習用とテスト用データを分割、ただし分割する際に_load_data()を適用
def train_test_split(df, test_size=0.1, n_prev=50):
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))
    ntrn = int(ntrn)
    X_train, y_train = _load_data(df.iloc[0:ntrn], n_prev)
    X_test, y_test = _load_data(df.iloc[ntrn:], n_prev)

    return (X_train, y_train), (X_test, y_test)



df_code_date_hourly = pd.read_csv('./code_run_num_hourly.csv')
split_date = lambda x: x.split(" ")[0]
df_code_date_hourly["date"] = list(map(split_date, df_code_date_hourly['timestamp'].values))


length_of_sequences = 24
test_size = 0.1

(_, _), (x, _) = train_test_split(df_code_date_hourly[['demand']], test_size=test_size,
                                                        n_prev=length_of_sequences)


## 推論エンドポイント呼び出し

In [96]:
exc[0]

array([[254],
       [269],
       [241],
       [115],
       [ 70],
       [ 63],
       [ 71],
       [ 16],
       [ 12],
       [ 40],
       [ 68],
       [164],
       [192],
       [259],
       [265],
       [148],
       [227],
       [730],
       [738],
       [912],
       [300],
       [271],
       [153],
       [147]])

In [93]:
from boto3 import Session
from sagemaker.predictor import csv_serializer
from sklearn.externals import joblib
import os
import argparse
import slackweb
import sagemaker
import pandas as pd
from sagemaker.tensorflow.predictor import _TFJsonDeserializer, _TFJsonSerializer
from sagemaker.tensorflow.serving import Predictor


endpoint="SageMakerTensorflowSaving"
profile = 'default'
session = Session(profile_name=profile)
sm_session = sagemaker.Session(boto_session=session)

predictor = Predictor(endpoint, sagemaker_session=sm_session)
predictor.serializer = _TFJsonSerializer()
predictor.deserializer = _TFJsonDeserializer()
predictor.content_type = "application/json"
predictor.accept = "application/json"



In [26]:
pred = predictor_ex.predict(exc[0])

In [27]:
pred

{'prediction': 165}

In [14]:
import json

json.dumps(exc[0].tolist())

'[[254], [269], [241], [115], [70], [63], [71], [16], [12], [40], [68], [164], [192], [259], [265], [148], [227], [730], [738], [912], [300], [271], [153], [147]]'

In [31]:
f = open("test.json")
# jsonデータを読み込んだファイルオブジェクトからPythonデータを作成
data = json.load(f)
data['threshold_value']

197355.8910006308

In [147]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

# inp = bytes(str(exc[0].flatten().tolist()), 'utf-8')
inp = json.dumps(exc[0].tolist())

response = client.invoke_endpoint(
    EndpointName='sagemaker-tensorflow-scriptmode-2019-11-08-04-10-40-842',
    Body=ops,
    ContentType='application/json',
    Accept='application/json'
)

body = response['Body']
json.load(body)

{'judgement': 1}